In [1]:
from finetuning_utils import *

Using cuda as DEVICE


In [2]:
import yaml
config_path = '/home/tu/tu_tu/tu_zxmav84/DS_Project/modules/config.yml'
with open(config_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
weights_dir = config['data']['model_weights']

### Get Loaders

In [3]:
train_loader, test_loader = get_munich_tuning_loaders(batch_size=2)

Indexing files in orthophotos/labeling_subset... 
Found: 	 150 Images 
	 150 Mask
Length of all files: 150
Length of train (120) and test (30): 150
Length of train loader: 60; Length of test loader: 15 with batch size 2


### Training

In [18]:
model_name = 'loveda_DeepLabV3_r101_epochs15_lr0-07487'
model = torch.load(weights_dir + '/own_training/' + model_name + '/' + model_name + '_epoch14.pth.tar', 
                map_location=torch.device('cpu'))
model = model.to(DEVICE)

#train_model(model, train_loader, test_loader, LEARNING_RATE=1e-2, NUM_EPOCHS = 2)

### Plot some predictions

In [19]:
inputs_and_preds = []
model.eval()  # set model to evaluation mode

with torch.no_grad():  # no need to track gradients
    for data, label in tqdm(test_loader):
        data = data.to(DEVICE)
        logits = model(data)['out']
        prob = torch.softmax(logits, dim=1)
        pred = torch.argmax(prob, dim=1)
        inputs_and_preds.append((data.cpu(), pred.cpu(), label.cpu()))



100%|██████████| 15/15 [00:01<00:00,  7.72it/s]


In [20]:
def denormalize(image, mean, std):
    image = image.clone().detach().numpy()  # Convert to numpy array
    image = image.transpose((1, 2, 0))  # Reshape (C, H, W) to (H, W, C)
    image = (image * std) + mean  # Denormalize
    image = image.clip(0, 1)  # Clip to range [0, 1]
    return image

def plot_random_images(inputs_preds_targets, idx2rgb, n):
    fig, ax = plt.subplots(n, 3, figsize=(15, 5*n))

    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])

    for i in range(n):
        # Select a random batch
        inputs, preds, targets = random.choice(inputs_preds_targets)
        
        # Select a random image from the batch
        idx = random.randint(0, len(inputs) - 1)
        image = inputs[idx]
        image = denormalize(image, mean, std)  # Denormalize the image

        pred = preds[idx].numpy()
        target = targets[idx].numpy()

        # Create RGB images according to the colormap
        pred_rgb = np.zeros((*pred.shape, 3), dtype=np.uint8)
        target_rgb = np.zeros((*target.shape, 3), dtype=np.uint8)
        for cls, color in idx2rgb.items():
            pred_rgb[pred == cls] = color
            target_rgb[target == cls] = color

        # Plot original image
        ax[i, 0].imshow(image)
        ax[i, 0].axis('off')
        ax[i, 0].set_title(f'Original Image {i+1}')

        # Plot target image
        ax[i, 1].imshow(target_rgb)
        ax[i, 1].axis('off')
        ax[i, 1].set_title(f'Target Image {i+1}')

        # Plot predicted image
        ax[i, 2].imshow(pred_rgb)
        ax[i, 2].axis('off')
        ax[i, 2].set_title(f'Predicted Image {i+1}')

    plt.tight_layout()
    plt.show()


In [24]:
import time
t = 1000 * time.time() # current time in milliseconds
np.random.seed(int(t) % 2**32)
plot_random_images(inputs_and_preds, idx2rgb, 10)